In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@etedatabricks01.dfs.core.windows.net/products")
df.display()

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE 
LANGUAGE SQL
RETURN round(p_price * 0.90, 2)

In [0]:
%sql
select product_id, price, databricks_catalog.bronze.discount_func(price) as discounted_price from products

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@etedatabricks01.dfs.core.windows.net/products")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.products_silver 
USING DELTA
LOCATION 'abfss://silver@etedatabricks01.dfs.core.windows.net/products'